### summarize elastic net results on Independent Simulation Scenarios for binary outcome

In [1]:
dir = '/panfs/panfs1.ucsd.edu/panscratch/lij014/Stability_2020/sim_data'

In [2]:
dim.list = list()
size = c(50, 100, 500, 1000)
idx = 0
for (P in size){
    for (N in size){
        idx = idx + 1
        dim.list[[idx]] = c(P=P, N=N)
    }
}

files = NULL
for (dim in dim.list){
    p = dim[1]
    n = dim[2]
    files = cbind(files, paste0(dir, '/sim_independent_', paste('P', p, 'N', n, sep='_'), '.RData'))
}

In [3]:
length(files)

[1] 16

In [4]:
avg_FDR = NULL
table_toe = NULL
tmp_num_select = rep(0, length(files))
for (i in 1:length(files)){
    print(paste0('indx: ', i))
    load(paste0(dir, '/binary_update/ind_Elnet_binary_', i, '.RData')) 
    
    table_toe = rbind(table_toe, results_ind_elnet[c('n', 'p', 'rou', 'FP', 'FN', 'ROC', 'Stab')])
    tmp_num_select[i] = mean(rowSums(results_ind_elnet$Stab.table))
    
    # calculate FDR
    load(file_name, dat <- new.env())
    sub = dat$sim_array[[i]]
    p = sub$p # take true values from 1st replicate of each simulated data
    coef = sub$beta
    coef.true = which(coef != 0)
    
    tt = results_ind_elnet$Stab.table
    FDR = NULL # false positive rate
    for (r in 1:nrow(tt)){
        FDR = c(FDR, length(setdiff(which(tt[r, ] !=0), coef.true))/sum(tt[r, ]))

    }
    
    avg_FDR = c(avg_FDR, mean(FDR, na.rm=T))
}
table_toe = as.data.frame(table_toe)
table_toe$num_select = tmp_num_select
table_toe$FDR = round(avg_FDR,2)

[1] "indx: 1"
[1] "indx: 2"
[1] "indx: 3"
[1] "indx: 4"
[1] "indx: 5"
[1] "indx: 6"
[1] "indx: 7"
[1] "indx: 8"
[1] "indx: 9"
[1] "indx: 10"
[1] "indx: 11"
[1] "indx: 12"
[1] "indx: 13"
[1] "indx: 14"
[1] "indx: 15"
[1] "indx: 16"


In [5]:
head(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
1,50,50,0,12.04 ( 0.89 ),1.42 ( 0.11 ),0.76 ( 0.01 ),0.15,15.62,0.61
2,100,50,0,11.44 ( 0.76 ),0.46 ( 0.07 ),0.84 ( 0.01 ),0.23,15.98,0.56
3,500,50,0,8.74 ( 0.72 ),0 ( 0 ),0.92 ( 0 ),0.36,13.74,0.45
4,1000,50,0,7.56 ( 0.72 ),0 ( 0 ),0.93 ( 0 ),0.41,12.56,0.40
5,50,100,0,18.92 ( 1.7 ),1.81 ( 0.12 ),0.72 ( 0.01 ),0.1,22.11,0.72
6,100,100,0,12.48 ( 0.9 ),0.96 ( 0.09 ),0.82 ( 0.01 ),0.22,16.52,0.58


In [6]:
tail(table_toe)

,n,p,rou,FP,FN,ROC,Stab,num_select,FDR
,<list>,<list>,<list>,<list>,<list>,<list>,<list>,<dbl>,<dbl>
11,500,500,0,16.79 ( 1.47 ),0 ( 0 ),0.92 ( 0 ),0.27,21.79,0.61
12,1000,500,0,11.47 ( 1.14 ),0 ( 0 ),0.93 ( 0 ),0.36,16.47,0.48
13,50,1000,0,72.88 ( 8.55 ),2.89 ( 0.13 ),0.67 ( 0.01 ),0.02,74.99,0.91
14,100,1000,0,29.95 ( 4.05 ),2 ( 0.1 ),0.76 ( 0.01 ),0.09,32.95,0.77
15,500,1000,0,19.47 ( 1.76 ),0 ( 0 ),0.91 ( 0 ),0.24,24.47,0.62
16,1000,1000,0,18.47 ( 1.76 ),0 ( 0 ),0.93 ( 0 ),0.25,23.47,0.58


In [7]:
# export result
result.table_toe <- apply(table_toe,2,as.character)
rownames(result.table_toe) = rownames(table_toe)
result.table_toe = as.data.frame(result.table_toe)

# extract numbers only for 'n' & 'p'
result.table_toe$n = tidyr::extract_numeric(result.table_toe$n)
result.table_toe$p = tidyr::extract_numeric(result.table_toe$p)
result.table_toe$ratio = result.table_toe$p / result.table_toe$n

result.table_toe = result.table_toe[c('n', 'p', 'rou', 'ratio', 'Stab', 'ROC', 'FP', 'FN', 'num_select', 'FDR')]
colnames(result.table_toe)[1:4] = c('N', 'P', 'Corr', 'Ratio')

extract_numeric() is deprecated: please use readr::parse_number() instead

extract_numeric() is deprecated: please use readr::parse_number() instead



In [8]:
# convert interested measurements to be numeric
result.table_toe$Stab = as.numeric(as.character(result.table_toe$Stab))
result.table_toe$num_select = as.numeric(as.character(result.table_toe$num_select))

result.table_toe$ROC_mean = as.numeric(sub("\\(.*", "", result.table_toe$ROC))
result.table_toe$FP_mean = as.numeric(sub("\\(.*", "", result.table_toe$FP))
result.table_toe$FN_mean = as.numeric(sub("\\(.*", "", result.table_toe$FN))

In [9]:
# check whether missing values exists
result.table_toe[rowSums(is.na(result.table_toe)) > 0,]

N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>


In [10]:
head(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.15,0.76 ( 0.01 ),12.04 ( 0.89 ),1.42 ( 0.11 ),15.62,0.61,0.76,12.04,1.42
2,100,50,0,0.50,0.23,0.84 ( 0.01 ),11.44 ( 0.76 ),0.46 ( 0.07 ),15.98,0.56,0.84,11.44,0.46
3,500,50,0,0.10,0.36,0.92 ( 0 ),8.74 ( 0.72 ),0 ( 0 ),13.74,0.45,0.92,8.74,0.00
4,1000,50,0,0.05,0.41,0.93 ( 0 ),7.56 ( 0.72 ),0 ( 0 ),12.56,0.4,0.93,7.56,0.00
5,50,100,0,2.00,0.10,0.72 ( 0.01 ),18.92 ( 1.7 ),1.81 ( 0.12 ),22.11,0.72,0.72,18.92,1.81
6,100,100,0,1.00,0.22,0.82 ( 0.01 ),12.48 ( 0.9 ),0.96 ( 0.09 ),16.52,0.58,0.82,12.48,0.96


In [11]:
tail(result.table_toe)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
11,500,500,0,1.0,0.27,0.92 ( 0 ),16.79 ( 1.47 ),0 ( 0 ),21.79,0.61,0.92,16.79,0.00
12,1000,500,0,0.5,0.36,0.93 ( 0 ),11.47 ( 1.14 ),0 ( 0 ),16.47,0.48,0.93,11.47,0.00
13,50,1000,0,20.0,0.02,0.67 ( 0.01 ),72.88 ( 8.55 ),2.89 ( 0.13 ),74.99,0.91,0.67,72.88,2.89
14,100,1000,0,10.0,0.09,0.76 ( 0.01 ),29.95 ( 4.05 ),2 ( 0.1 ),32.95,0.77,0.76,29.95,2.00
15,500,1000,0,2.0,0.24,0.91 ( 0 ),19.47 ( 1.76 ),0 ( 0 ),24.47,0.62,0.91,19.47,0.00
16,1000,1000,0,1.0,0.25,0.93 ( 0 ),18.47 ( 1.76 ),0 ( 0 ),23.47,0.58,0.93,18.47,0.00


In [12]:
result.table_toe

## export
write.table(result.table_toe, '../results_summary_bin/sim_ind_Elnet_binary.txt', sep='\t', row.names=F)

,N,P,Corr,Ratio,Stab,ROC,FP,FN,num_select,FDR,ROC_mean,FP_mean,FN_mean
,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>
1,50,50,0,1.00,0.15,0.76 ( 0.01 ),12.04 ( 0.89 ),1.42 ( 0.11 ),15.62,0.61,0.76,12.04,1.42
2,100,50,0,0.50,0.23,0.84 ( 0.01 ),11.44 ( 0.76 ),0.46 ( 0.07 ),15.98,0.56,0.84,11.44,0.46
3,500,50,0,0.10,0.36,0.92 ( 0 ),8.74 ( 0.72 ),0 ( 0 ),13.74,0.45,0.92,8.74,0.00
4,1000,50,0,0.05,0.41,0.93 ( 0 ),7.56 ( 0.72 ),0 ( 0 ),12.56,0.4,0.93,7.56,0.00
5,50,100,0,2.00,0.10,0.72 ( 0.01 ),18.92 ( 1.7 ),1.81 ( 0.12 ),22.11,0.72,0.72,18.92,1.81
6,100,100,0,1.00,0.22,0.82 ( 0.01 ),12.48 ( 0.9 ),0.96 ( 0.09 ),16.52,0.58,0.82,12.48,0.96
7,500,100,0,0.20,0.32,0.92 ( 0 ),11.78 ( 0.99 ),0 ( 0 ),16.78,0.52,0.92,11.78,0.00
8,1000,100,0,0.10,0.34,0.93 ( 0 ),10.71 ( 1.03 ),0 ( 0 ),15.71,0.47,0.93,10.71,0.00
9,50,500,0,10.00,0.03,0.68 ( 0.01 ),57.17 ( 5.72 ),2.27 ( 0.13 ),59.90,0.88,0.68,57.17,2.27
